#  N dismsional pipeline example

In [ ]:
from makePDFMaps import *
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from bokeh.io import output_notebook
import matplotlib.pyplot as plt
output_notebook()

In [ ]:
# Parameters:
sigmaPoint=0.3
npoints=4000000

# Random Data Frame generator


* Generate random data with spread sigmaPoint in 4 D space
  * $ y=A\, e^{B} \sin{C} $

In [ ]:
df = pd.DataFrame(np.random.random_sample(size=(npoints, 3)), columns=list('ABC'))
df["noise"]=np.random.normal(0, 0.15, npoints)
df["csin"]=np.sin(6.28*df["C"])
df["valueOrig"]=df["A"]+np.exp(0.0*df["B"])*df["csin"]
df["value"]=df["valueOrig"]+df["noise"]
df.tail()


# Make Histogram

* make 5D histogram
  * using ./RootInteractive/Tools/histoNDTools.py

In [ ]:
histo=makeHistogram(df,"value:A:B:C:#A>0>>myhisto(40,-2,2,50,0,1,100,0,1,50,0,1)")


# Inputs

* make statistic maps 

In [ ]:
slices=((0,40,1,0),(10,30,1,2),(10,24,1,2),(1,49,3,1))   # start:stop:step:width
dimI=0                                                               # dimension of interest  

In [ ]:
dframe = makePdfMaps(histo,slices,dimI, quantiles=[25,50,75])
dframe.sample(10)

* compare extracted statistic with the original function
  * residula histogram - stat - point
    ** rms should be ~ simgaPoint
  * residual hsitogram - statt - original function
    ** rms should be ~/sigmaPoint/sqrt(nPoints) per bin
  * values will be close to expectation in case sufficiennt number of bins
    * Sampling should be more than 1 sample per sigma (https://en.wikipedia.org/wiki/Nyquist%E2%80%93Shannon_sampling_theorem)
    * second derivative within bin should be negligible
    * for mean sigmaPoint>dvalue/dXidXj *dxidxj
    * for sigma  sigmaPoint>dvalue/dXi *dxi
    
    

In [ ]:
dframe["csin"]=np.sin(6.28*dframe["CBinCenter"])
dframe["valueOrig"]=dframe["ABinCenter"]+np.exp(0*dframe["BBinCenter"])*dframe["csin"]
dframe["stddev"]=np.sqrt(dframe["rms"]**2-dframe["means"]**2)
dframe["quantile_diff"]=dframe["quantile_75"] - dframe["quantile_25"]
dframe["median_diff"]=dframe["medians"] - dframe["quantile_50"]
dframe.sample(20)


In [ ]:
dframe["median_diff"].hist()

In [ ]:
plt.plot(dframe["stddev"])
plt.show()

In [ ]:

figureArray = [
    [['CBinCenter'], ['valueOrig','means-valueOrig', 'medians-valueOrig']],
    [['CBinCenter'],['quantile_diff']]
]
tooltips = [("A", "(@ABinCenter)"),("B", "(@BBinCenter)"),("C", "(@CBinCenter)"),("D", "(@DBinCenter)"), ]
figureLayout: str = '((0),(1),x_visible=1,y_visible=0,plot_height=250,plot_width=1000)'
bokehDrawSA.fromArray(dframe, "ABinNumber>0",figureArray, [], layout=figureLayout, tooltips=tooltips  )

In [ ]:
dframe["diff"]=dframe["valueOrig"]-dframe["means"]
dframe["pull"]=dframe["diff"]/(sigmaPoint/np.sqrt(dframe["entries"]))
dframe.head(5)
dframe["diff"].hist(bins=100)
dframe["pull"].hist(bins=100)
dframe["diff"].std()
dframe["pull"].std()

In [ ]:
bokehDraw?